# **Introducción al análisis de datos en Python** 
#### Profesora: Catalina Bernal

### Unir bases de datos (`merge`)

Ya aprendimos cómo concatenar filas y columnas de diferentes bases de datos. Para hacer esto es necesario que la cantidad de columnas y filas, respectivamente, de los dataframes a juntar sean los mismos y que sus índices o llaves también lo sean.

No obstante, muchas veces cuando trate de juntar bases de datos, notará que no necesariamente todas las llaves están presentes en ambas bases de datos, o que incluso, a cada fila de la base izquierda, querrá pegarle más de una fila de la base derecha, o viceversa.

A la hora de hacer pegues más complejos, hablamos de que vamos a utilizar un `merge`. 

Comencemos con la sintaxis del `merge`. Para pegar dos bases de datos, usted usará un comando similar al siguiente:

```python
pd.merge(left = left_dataframe, right = right_dataframe, on = "alguna(s)_columa(s)", how = "left|right|inner|outer")
```

Los argumentos que toma la función son:
- `left`: dataframe que va de primero.
- `right`: dataframe que va de segundo.
- `on`: es la columna o la lista de columnas que determinan qué filas de una tabla coinciden con qué filas de la segunda tabla. Comúnmente a estas variables se les llaman las llaves del pegue y debe identificar a cada observación de forma única. A veces, las columnas que desea fusionar tienen nombres diferentes en los datos. Por ejemplo, suponga que tiene dos bases de datos, una que registra el dinero mensual gastado por persona en almacenes Éxito y otra que tiene características personales de las personas. Usted podría tratar de juntar ambas bases con el identificador de fila o persona de cada base que en este caso podría ser la cédula, sin embargo, en un dataframe tal vez la variable se llame "cc" mientras que en el otro puede que se llame "cédula". En esos casos, puede especificar los nombres de columna por separado para cada marco de datos utilizando los argumentos "left_on" y "right_on".
- `how`: es el método a usar, por defecto Pandas usa el método "inner". Más adelante exploraremos más al respecto.

<center>
<div>
<img src="./img/merges.png" width="400"/>
</div>
</center>

Tenemos cuatro grandes métodos para relacionar las bases porque no siempre tenemos una coincidencia uno a uno (one to one) entre las filas. Estos cuatro métodos afectan la forma en que Pandas trata los datos no coincidentes y eso es lo que veremos más adelante. ```validate```

<center>
<div>
<img src="./img/one-many.png" width="400"/>
</div>
</center>



In [1]:
import pandas as pd
import numpy as np

In [9]:
# ejemplos de pegues
left_dataframe = pd.DataFrame({"ID": [1,2,3,4], "left_side": "Izquierda"})
right_dataframe = pd.DataFrame({"ID": [3,4,3,6], "right_side": "Derecha"})

In [10]:
left_dataframe

,ID,left_side
0,1,Izquierda
1,2,Izquierda
2,3,Izquierda
3,4,Izquierda


In [11]:
right_dataframe

,ID,right_side
0,3,Derecha
1,4,Derecha
2,3,Derecha
3,6,Derecha


#### Left merge
En un Left merge lo que más nos interesa son los datos del lado IZQUIERDO a los cuales queremos pegarles columnas de una base de datos en el lado DERECHO.

Para hacer eso, cortamos las filas en el marco de datos DERECHO y pegamos partes en el marco de datos IZQUIERDO. Recuerde, nos preocupamos principalmente por el lado IZQUIERDO y solo queremos datos del lado DERECHO si tiene alguna de las mismas ID. Entonces, si algo en el marco de datos DERECHO no coincide o no existe, entonces tenemos que hacer cosas para mantener las columnas de la misma longitud. Lo hacemos agregando NaN para llenar el vacío o descartando algunas filas por completo.

En este ejemplo, el lado IZQUIERDO tiene los ID 1, 2, 3 y 4:
- El lado DERECHO no tiene ID 1 o 2, por lo que agregamos NaN porque necesitamos que las columnas tengan la misma longitud.
- El lado DERECHO tiene datos para los ID 3 y 4, así que lo agregamos como una nueva columna.
- El lado IZQUIERDO no tiene ID 5 o 6, por lo que no necesitamos esa información del DERECHO y se descarta.

<center>
<div>
<img src="./img/left_merge.png" width="400"/>
</div>
</center>

In [12]:
# Left merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "left")

,ID,left_side,right_side
0,1,Izquierda,NaN
1,2,Izquierda,NaN
2,3,Izquierda,Derecha
3,3,Izquierda,Derecha
4,4,Izquierda,Derecha


In [16]:
left_dataframe.merge(right_dataframe, how = "left", validate="1:m", indicator=True)

,ID,left_side,right_side,_merge
0,1,Izquierda,NaN,left_only
1,2,Izquierda,NaN,left_only
2,3,Izquierda,Derecha,both
3,3,Izquierda,Derecha,both
4,4,Izquierda,Derecha,both


#### Right merge
Los Right merges funcionan igual que los Left merges, la diferencia es que nos preocupamos principalmente por el lado DERECHO y nos gustaría agregar datos desde el IZQUIERDO si tienen ID coincidentes.

<center>
<div>
<img src="./img/right_merge.png" width="400"/>
</div>
</center>

In [18]:
# Right merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "right", indicator=True)

,ID,left_side,right_side,_merge
0,3,Izquierda,Derecha,both
1,4,Izquierda,Derecha,both
2,3,Izquierda,Derecha,both
3,6,NaN,Derecha,right_only


#### Inner merge
Con un Inner merge, cortamos ambos marcos de datos y solo pegamos las cosas que coinciden. Si una ID no está en ambos marcos de datos, no la mantenemos y no agregamos NaN.

<center>
<img src="./img/inner_merge.png" width="400"/>
</center>

In [20]:
# Inner merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "inner", indicator=True)

,ID,left_side,right_side,_merge
0,3,Izquierda,Derecha,both
1,3,Izquierda,Derecha,both
2,4,Izquierda,Derecha,both


#### Outer merge
Con un Outer merge, cortamos ambos marcos de datos y mantenemos todo de ambos lados. Luego agregamos NaN para llenar los espacios en blanco.

<center>
<img src="./img/outer_merge.png" width="400"/>
</center>

In [21]:
# Outer merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "outer", indicator=True)

,ID,left_side,right_side,_merge
0,1,Izquierda,NaN,left_only
1,2,Izquierda,NaN,left_only
2,3,Izquierda,Derecha,both
3,3,Izquierda,Derecha,both
4,4,Izquierda,Derecha,both
5,6,NaN,Derecha,right_only


In [29]:
# Creamos dos DataFrames: empleados y departamentos
empleados = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'nombre': ['Ana', 'Luis', 'Carlos', 'Sofía'],
    'num_departamento': [10, 20, 10, 30],
    'categoria': ['A', 'B', 'A', 'C']
})
empleados

,id,nombre,num_departamento,categoria
0,1,Ana,10,A
1,2,Luis,20,B
2,3,Carlos,10,A
3,4,Sofía,30,C


In [30]:
departamentos = pd.DataFrame({
    'departamento_id': [10, 20, 30],
    'departamento': ['Ventas', 'Marketing', 'TI'],
    'categoria': ['A', 'B', 'C']
})

departamentos

,departamento_id,departamento,categoria
0,10,Ventas,A
1,20,Marketing,B
2,30,TI,C


In [32]:
# Unimos usando la columna en común
df_unido = empleados.merge(departamentos, left_on='num_departamento', right_on='departamento_id', indicator=True, suffixes=('_empleado', '_departamento'))
df_unido

,id,nombre,num_departamento,categoria_empleado,departamento_id,departamento,categoria_departamento,_merge
0,1,Ana,10,A,10,Ventas,A,both
1,2,Luis,20,B,20,Marketing,B,both
2,3,Carlos,10,A,10,Ventas,A,both
3,4,Sofía,30,C,30,TI,C,both


In [ ]:
for i in range(len(df_unido)):
    empleados['nombre'][i][0] in 'aieu'

False

In [38]:
# Queremos saber si el nombre de la persona empieza con vocal
def empieza_con_vocal(nombre):
    return nombre[0].lower() in 'aeiou'

empleados['empieza_vocal'] = empleados['nombre'].apply(empieza_con_vocal)
empleados

,id,nombre,num_departamento,categoria,empieza_vocal
0,1,Ana,10,A,True
1,2,Luis,20,B,False
2,3,Carlos,10,A,False
3,4,Sofía,30,C,False


### Groupby


Uno de los métodos más útiles para los analistas de datos es `.groupby()`. Este método permite dividir los datos en grupos y a cada uno de estos aplicarles una función de agregación.

Veamos el siguiente ejemplo para entender este concepto mejor:

In [39]:
df = pd.read_excel(f"data/ejemplo_groupby.xlsx")
df

,animal,age,weight,length
0,hamster,1,7,8
1,alligator,9,13,6
2,hamster,4,8,9
3,cat,13,12,1
4,snake,14,11,8
5,cat,10,8,9
6,hamster,2,10,5
7,cat,4,14,6
8,cat,14,9,6
9,snake,7,11,6


In [40]:
df.animal.unique()

array(['hamster', 'alligator', 'cat', 'snake'], dtype=object)

Note que tenemos un `dataframe` con cuatro tipos de animales: 
- alligators (cocodrilos 🐊)
- cats (gatos 🐱)
- snakes (serpientes 🐍)
- hamsters (hamsters 🐹)

Cada una de las filas indican un chequeo en el veterinario donde se registra edad, peso y largo del animal. Por ende, usted como investigador quiere estudiar algunas estadísticas descriptivas por especie. Por ejemplo ¿Cuál es el peso promedio de cada especie?

In [41]:
# El primer paso es agrupar por animal
animal_groups = df.groupby("animal")

In [42]:
animal_groups

In [43]:
# Veamos la conformación de cada uno de los grupos. ¿En qué filas aparece cada animal?
animal_groups.groups

{'alligator': [1, 13], 'cat': [3, 5, 7, 8, 12], 'hamster': [0, 2, 6, 10, 11], 'snake': [4, 9]}

Visualmente, lo que sucedió fue lo siguiente:

1. Se agrupa los valores únicos de la columna animal.
<center>
<img src = "./img/groupby1.jpg" width = "400">
</center>

2. La segmentación de cada grupo se vería de la siguiente manera
<center>
<img src = "./img/groupby2.jpg" width = "400">
</center>

3. Se le asignan las otras variables/columnas a cada grupo
<center>
<img src = "./img/groupby3.jpg" width = "400">
</center>

4. Se aplica la función agregadora `.mean()` sobre la columna `weight` de cada grupo.
<center>
<img src = "./img/groupby4.jpg" width = "400">
</center>


In [ ]:
df

#### Método .agg()
El método .agg() se puede utilizar después de aplicar un método .groupby() en pandas para realizar operaciones de agregación en los datos de cada grupo.

La sintaxis general de la función .groupby() es la siguiente:
```python
dataframe.groupby(columnas).agg(funciones)
```
Donde:
- dataframe: el DataFrame al que se aplicará la función `groupby()`.
- columnas: la(s) columna(s) que se utilizarán para agrupar los datos.
- funciones: la(s) operación(es) de agregación que se aplicarán a los datos agrupados.

Por ejemplo, para calcular la media, el máximo y el mínimo de las columnas de peso y longitud del DataFrame agrupado por la columna 'animal', se puede utilizar la siguiente sintaxis:

In [44]:
df.groupby("animal")[['weight','length']].agg(["min", "mean", "max"])

weight           length         
             min  mean max    min mean max
animal                                    
alligator     13  13.5  14      5  5.5   6
cat            8  10.4  14      1  5.2   9
hamster        7   9.0  10      3  6.0   9
snake         11  11.0  11      6  7.0   8

In [45]:
df.groupby("animal").agg({'weight': ['mean', 'max'], 'length': 'std', 
                                     "age": lambda x: np.percentile(x, 50)})

weight        length      age
            mean max       std <lambda>
animal                                 
alligator   13.5  14  0.707107      8.0
cat         10.4  14  2.949576     10.0
hamster      9.0  10  2.449490      2.0
snake       11.0  11  1.414214     10.5

In [46]:
# Otra sintaxis, en vez de un diccionario, usar tuplas
# (nombre_columna,funcion)
df.groupby("animal").agg(peso_promedio = ("weight", 'mean'), 
                                   peso_maximo = ("weight", 'max'),
                                   edad_mediana = ("age", lambda x: np.percentile(x, 50)))

,peso_promedio,peso_maximo,edad_mediana
animal,,,
alligator,13.5,14,8.0
cat,10.4,14,10.0
hamster,9.0,10,2.0
snake,11.0,11,10.5


## Ejercicio

### Información de accidentes de tránsito en Bogotá en el 2016
A partir de la base de datos `info_accidentes.csv`, responda las siguientes preguntas:

1. ¿Cuántos muertos hay registrados en la base de datos?
2. ¿Cuántos heridos hay registrados en la base de datos?
3. ¿Cuantos accidentes hubo en la localidad de Santa Fe? ¿Cuántos muertos y cuántos heridos dejaron estos accidentes en total?
4. Explore la columna TipoTiempo, ¿cuál es la categoría en la que más ocurren accidentes?

In [53]:
df = pd.read_csv('data/info_accidentes.csv')
df.head()

,Fecha,GravedadNombre,ClaseNombre,ChoqueNombre,ObjetoFijoCodigo,ObjetoFijoNombre,OtraClase,NombreOtraClase,Latitud,Longitud,...,TipoVia2,NumeroVia2,LetraVia2,CardinalVia2,Localidad,HoraOcurrencia,TipoDiseño,TipoTiempo,TotalMuertos,TotalHeridos
0,01/13/2016 12:00:00 AM,Con Heridos,Choque,Vehiculo,,,,,0.0,0.0,...,CL,83.0,,,ENGATIVA,12/31/1899 10:30:00 PM,Interseccion,Normal,0,2
1,01-12-16 0:00,Con Heridos,Atropello,,,,,,0.0,0.0,...,KR,7.0,,,USAQUEN,12/31/1899 03:40:00 PM,Interseccion,Normal,0,1
2,01/13/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,KR,5.0,,E,SAN CRISTOBAL,12/31/1899 07:20:00 AM,Tramo de Via,Normal,0,0
3,01/17/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,KR,107.0,,,ENGATIVA,12/31/1899 06:10:00 AM,Tramo de Via,Normal,0,0
4,01/18/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,CL,48.0,,S,RAFAEL URIBE URIBE,12/31/1899 05:00:00 PM,Tramo de Via,Normal,0,0


In [57]:
df.Localidad == "SANTA FE"

0        False
1        False
2        False
3        False
4        False
         ...  
34926    False
34927    False
34928    False
34929    False
34930    False
Name: Localidad, Length: 34931, dtype: bool

In [ ]:
df.loc[df.Localidad == "SANTA FE"]['TotalHeridos'].sum()

np.int64(439)

In [60]:
df.TipoTiempo

0        Normal
1        Normal
2        Normal
3        Normal
4        Normal
          ...  
34926    Normal
34927    Normal
34928    Normal
34929    Normal
34930    Normal
Name: TipoTiempo, Length: 34931, dtype: object

In [62]:
df.TipoTiempo.unique()

array(['Normal', 'Lluvia', 'Viento', 'Niebla', ' ', 'Lluvia/Lluvia',
       'Normal/Normal', 'Lluvia/Normal', 'Normal/Lluvia', 'Viento/Normal'],
      dtype=object)

In [61]:
df.TipoTiempo.value_counts()

TipoTiempo
Normal           33607
Lluvia            1214
Viento              43
Niebla              35
                    10
Normal/Normal       10
Lluvia/Normal        7
Lluvia/Lluvia        2
Normal/Lluvia        2
Viento/Normal        1
Name: count, dtype: int64